In [34]:
import os
import logging
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import torch as th
from torch.utils.data import DataLoader, Dataset

from transformers import TrainingArguments, RobertaForSequenceClassification

from clearml import InputModel

from utils import prepare_grader_data, compute_metrics

In [40]:

# Define constants
SUDY_NUMBER = 1

data_used = "famous_and_semi"

models_names = [
    "e0eab6ffd27b4b4dbedcc5c435f18742"
]
# Set up environment
trained_models_path = f"./anon_grader/trained_models/"
data_dir = f"textwash_data/study{SUDY_NUMBER}/intruder_test/full_data_study.csv"
results_dir = "./anon_grader"

DEVICE = "cuda" if th.cuda.is_available() else "cpu"

logging.info(f'Working on device: {DEVICE}')

# Cancel wandb logging
os.environ["WANDB_DISABLED"] = "true"

# Set seeds
SEED = 42
np.random.seed(SEED)
th.manual_seed(SEED)


In [41]:
# Read the data
columns_to_read = ["type", "text", "file_id", "name", "got_name_truth_q2"]
raw_data = pd.read_csv(data_dir, usecols=columns_to_read)

# Aggregate by file_id and calculate the rate of re-identification
data = (
    raw_data.groupby(["type", "file_id", "name", "text"])
    .agg({"got_name_truth_q2": "mean"})
    .reset_index()
)
data.rename(columns={"got_name_truth_q2": "human_rate"}, inplace=True)

# Define population to use
data = data[data["type"].isin(["famous", "semifamous"])]

# Preprocess the data
test_dataset = prepare_grader_data(data, SEED, DEVICE)['test']

# Create a DataLoader for the test dataset
test_dataloader = DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=False,
    num_workers=4,
)

In [42]:
# Predict with all models
for model_name in models_names:

    # Load the model
    logging.info(f"Loading model from {model_name}")

    model = InputModel(model_name)
    print(model)
#     # model.eval()

#     # Prediction
#     predictions = []
#     for batch in test_dataloader:
#         with th.no_grad():
#             regression_values = model(**batch).logits.squeeze().cpu().tolist()
#         predictions.extend(regression_values)

#     # Add predictions to the data
#     data[f"model_{file}"] = predictions

# # Calculate the overall mse for each model
# results = {
#     model_name: compute_metrics((data[model_name], data["human_rate"]))["mse"]
#     for model_name in data.columns[7:]
# }

In [44]:
model.get_local_copy()

Action failed <400/201: models.get_by_id/v1.0 (Invalid model id (no such public or company model): id=e0eab6ffd27b4b4dbedcc5c435f18742, company=e1221b134f224b5aa371e46e1e28ce62)> (model=e0eab6ffd27b4b4dbedcc5c435f18742)
Failed reloading task e0eab6ffd27b4b4dbedcc5c435f18742


AttributeError: 'NoneType' object has no attribute 'system_tags'